# Linear Solver Performance: 2D Stokes manufactured Solution after Botti and Di Pietro
### Part 1, Benchmark Setup and Execution

This benchmark was proposed in a work:

 "p‑Multilevel Preconditioners for HHO Discretizations of the Stokes Equations with Static Condensation" 
 by L. Botti and D. Di Pietro (https://doi.org/10.1007/s42967-021-00142-5)
 
It is used to assess DG as well as HDG methods, the latter beeing the main focus of the paper, the former only for reference.
The exact solution to the stationary Stokes equation is
$$
u_1 =  -\exp(x)*(y*\cos(y) + \sin(y)), \\
u_2 =  \exp(x)*y*\sin(y), \\
  p =  2*\exp(x)*\sin(y) .
$$
The domain is set as $ \Omega = (-1,1) $.
This exact solution is also used as a boundary condition on three sides; On one side, a Neumann boundary condition is enforced. It is not mentioned which side this is. 


### Note

This example can be found in the source code repository as as `LinslvPerf_BottiPietroStokes2D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 5/16/2022 11:59:38 AM


In [2]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
wmg.Init("LinslvPerfSer");
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

Project name is set to 'LinslvPerfSer'.
Opening existing database '\\fdygitrunner\ValidationTests\LinslvPerfSer'.


In [4]:
// for examination on the local workstation, 
//OpenDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes");

In [5]:
/*
// extract the control object to reproduce specific simulation on local workstation
string dest = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\BenchControls";
foreach(var s in wmg.Sessions) {
    string name = s.Name;
    Console.Write(name + ": ");
    try {
        var ctrl = s.GetControl();
        var txt = ctrl.Serialize();
        
        string DestPath = System.IO.Path.Combine(dest, name + ".obj");
        System.IO.File.WriteAllText(DestPath, txt);
        
        Console.WriteLine("written");
    } catch(Exception e) {
        Console.WriteLine(e.Message);
    }
}
*/

In [6]:
//wmg.DefaultDatabase.Grids.ForEach(s => s.Delete(true));

## Utility definitions

In [7]:
static class Utils {
    // DOF per cell for one variable
    public static int Np(int p) {
        return (p*p + 3*p + 2)/2; // 2D 
        //return (p*p*p + 6*p*p + 11*p + 6)/6; // 3D
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

In [8]:
int[] Resolutions_2D = new int[] { 8, 16, 24, 32, 48, 64, 128, 192, 256, 384, 512 };

IGridInfo[] grids = new IGridInfo[Resolutions_2D.Length];
for(int cnt = 0; cnt < Resolutions_2D.Length; cnt++) {
    int Res = Resolutions_2D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "-Stokes2D_J" + J);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(GridName)); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid2D.Cartesian2DGrid(xNodes, yNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            if(Math.Abs(x - (-1)) < 1e-8)
                return "pressure_outlet";
            return "wall"; 
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 2)
            throw new Exception("D mismatch");
    }
}

Opening existing database 'C:\BoSSStests\LinslvPerfSer'.
Found Grid: { Guid = 71453b0b-2b9f-456c-bd49-f5cf994aea67; Name = LinslvPerfSer-Stokes2D_J64; Cell Count = 64; Dim = 2 }
Found Grid: { Guid = d8d89ac6-7a24-4e66-b2a1-f6b037ea96a3; Name = LinslvPerfSer-Stokes2D_J256; Cell Count = 256; Dim = 2 }
Found Grid: { Guid = 485caa82-896f-4674-8958-ae296d9b1d42; Name = LinslvPerfSer-Stokes2D_J576; Cell Count = 576; Dim = 2 }
Found Grid: { Guid = ba6ce9a4-f41a-40bc-83ae-2ba15dff4f9a; Name = LinslvPerfSer-Stokes2D_J1024; Cell Count = 1024; Dim = 2 }
Found Grid: { Guid = bfee9062-773f-4cbf-b20b-be749c3943b7; Name = LinslvPerfSer-Stokes2D_J2304; Cell Count = 2304; Dim = 2 }
Found Grid: { Guid = 02a2ed43-b06b-43e7-ae3c-b6d66c1e741a; Name = LinslvPerfSer-Stokes2D_J4096; Cell Count = 4096; Dim = 2 }
Found Grid: { Guid = 8e3f6b09-5703-4464-b850-400630e49880; Name = LinslvPerfSer-Stokes2D_J16384; Cell Count = 16384; Dim = 2 }
Found Grid: { Guid = f4a19a12-1747-4340-a347-f96f7b8f9bce; Name = LinslvPe

In [9]:
grids

#0: { Guid = 71453b0b-2b9f-456c-bd49-f5cf994aea67; Name = LinslvPerfSer-Stokes2D_J64; Cell Count = 64; Dim = 2 }
#1: { Guid = d8d89ac6-7a24-4e66-b2a1-f6b037ea96a3; Name = LinslvPerfSer-Stokes2D_J256; Cell Count = 256; Dim = 2 }
#2: { Guid = 485caa82-896f-4674-8958-ae296d9b1d42; Name = LinslvPerfSer-Stokes2D_J576; Cell Count = 576; Dim = 2 }
#3: { Guid = ba6ce9a4-f41a-40bc-83ae-2ba15dff4f9a; Name = LinslvPerfSer-Stokes2D_J1024; Cell Count = 1024; Dim = 2 }
#4: { Guid = bfee9062-773f-4cbf-b20b-be749c3943b7; Name = LinslvPerfSer-Stokes2D_J2304; Cell Count = 2304; Dim = 2 }
#5: { Guid = 02a2ed43-b06b-43e7-ae3c-b6d66c1e741a; Name = LinslvPerfSer-Stokes2D_J4096; Cell Count = 4096; Dim = 2 }
#6: { Guid = 8e3f6b09-5703-4464-b850-400630e49880; Name = LinslvPerfSer-Stokes2D_J16384; Cell Count = 16384; Dim = 2 }
#7: { Guid = f4a19a12-1747-4340-a347-f96f7b8f9bce; Name = LinslvPerfSer-Stokes2D_J36864; Cell Count = 36864; Dim = 2 }
#8: { Guid = 708a43c5-ada6-4591-af5e-3cb12ad67969; Name = LinslvPerf

In [10]:
//PlotGrid("g2304",grids[4]);

In [11]:
wmg.DefaultDatabase

{ Session Count = 242; Grid Count = 30; Path = \\fdygitrunner\ValidationTests\LinslvPerfSer }

## Setup Control Object for a Solver Run

In [12]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [13]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Boundary Conditions and Exact Solution

In [14]:
var VelocityX = new Formula("(X) => -Math.Exp(X[0])*(X[1]*Math.Cos(X[1]) + Math.Sin(X[1]))");

In [15]:
var VelocityY = new Formula("(X) => Math.Exp(X[0])*X[1]*Math.Sin(X[1])");

In [16]:
var Pressure = new Formula("(X) => 2*Math.Exp(X[0])*Math.Sin(X[1])");

In [17]:
NUnit.Framework.Assert.Less((VelocityX.Evaluate(new double[] { 1, 0.5 }, 0) - (-2.495972095)).Abs(), 1.0e-9, 
     "x-Velocity expression differs from reference value");

In [18]:
NUnit.Framework.Assert.Less((VelocityY.Evaluate(new double[] { 1, 0.5 }, 0) - 0.6516068645).Abs(), 1.0e-9, 
     "y-Velocity expression differs from reference value");

### Setup of Parameter Study

Polynomial degrees to test:

In [19]:
int[] PolyDegS = new int[] {2, 3, 5};

Solvers which we want to instrument:

In [20]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.direct_pardiso,
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.pMultigrid
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [21]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.direct_pardiso:
        case LinearSolverCode.direct_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 3000000; // Up to 3 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [22]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [23]:
List<XNSE_Control> controls = new List<XNSE_Control>();
controls.Clear();
foreach(LinearSolverCode solver in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {

    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*2 + Np_p;
    if(J*DOF >  GetMaxAllowedDOF(solver))
        continue;
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add(C);
       
    string caseName = string.Format("BottiPietroStokes2D-J{0}_p{1}_{2}", J, k, solver);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    C.PhysicalParameters.rho_A             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.rho_B             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.mu_A              = 1; // dimensionless
    C.PhysicalParameters.mu_B              = 1; // dimensionless
    C.PhysicalParameters.Sigma             = 0; // not relevant, since single phase
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
    // Boundary Conditions
    // ===================
    C.AddBoundaryValue("wall", "VelocityX", VelocityX);
    C.AddBoundaryValue("wall", "VelocityY", VelocityY);
    
    C.AddInitialValue("VelocityX", VelocityX);
    C.AddInitialValue("VelocityY", VelocityY);
   
    // Solver Stuff
    // ============
    
    //C.VelocityBlockPrecondMode         = MultigridOperator.Mode.SymPart_DiagBlockEquilib;
    
    C.LinearSolver           = solver.GetConfig();
    if(C.LinearSolver is IterativeSolverConfig isc) {
        isc.ConvergenceCriterion = 1e-8;
    }
    C.LevelSet_ConvergenceCriterion     = 1e-6;
    C.NoOfMultigridLevels = 100;

    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
}
}
}

setting up: BottiPietroStokes2D-J64_p2_direct_pardiso
setting up: BottiPietroStokes2D-J256_p2_direct_pardiso
setting up: BottiPietroStokes2D-J576_p2_direct_pardiso
setting up: BottiPietroStokes2D-J1024_p2_direct_pardiso
setting up: BottiPietroStokes2D-J2304_p2_direct_pardiso
setting up: BottiPietroStokes2D-J4096_p2_direct_pardiso
setting up: BottiPietroStokes2D-J16384_p2_direct_pardiso
setting up: BottiPietroStokes2D-J36864_p2_direct_pardiso
setting up: BottiPietroStokes2D-J65536_p2_direct_pardiso
setting up: BottiPietroStokes2D-J64_p3_direct_pardiso
setting up: BottiPietroStokes2D-J256_p3_direct_pardiso
setting up: BottiPietroStokes2D-J576_p3_direct_pardiso
setting up: BottiPietroStokes2D-J1024_p3_direct_pardiso
setting up: BottiPietroStokes2D-J2304_p3_direct_pardiso
setting up: BottiPietroStokes2D-J4096_p3_direct_pardiso
setting up: BottiPietroStokes2D-J16384_p3_direct_pardiso
setting up: BottiPietroStokes2D-J36864_p3_direct_pardiso
setting up: BottiPietroStokes2D-J64_p5_direct_pardi

Total number of simulations:

In [24]:
controls.Count

105

In [25]:
// Basic checks on the multigrid configuration
foreach(var ctrl in controls) {
    NUnit.Framework.Assert.Greater(ctrl.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
    if(ctrl.LinearSolver is OrthoMGSchwarzConfig osc)
        NUnit.Framework.Assert.Greater(osc.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
}

In [26]:
//string path = @"C:\Users\jenkinsci\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\Stokes2D";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, "control-" + ctrl.SessionName + ".obj"));
//}

## Launch Jobs

In [27]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    ctrl.RunBatch();
}

 Submitting: BottiPietroStokes2D-J64_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J64_p2_direct_pardiso	04/15/2022 09:13:55	15afafd8..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J256_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J256_p2_direct_pardiso	04/15/2022 09:14:31	5e6d305d..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J576_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J576_p2_direct_pardiso	04/15/2022 09:15:06	97e38d09..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J1024_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J1024_p2_direct_pardi

No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J4096_p2_exp_gmres_levelpmg
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J4096_p2_exp_gmres_levelpmg	04/15/2022 09:41:49	ac3bdf24..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J16384_p2_exp_gmres_levelpmg
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J16384_p2_exp_gmres_levelpmg	04/15/2022 09:43:18	1230fdf5..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J36864_p2_exp_gmres_levelpmg
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J36864_p2_exp_gmres_levelpmg	04/15/2022 09:44:47	758a1841..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J65536_p2_exp_gmre

No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J16384_p2_exp_Kcycle_schwarz
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J16384_p2_exp_Kcycle_schwarz	04/15/2022 10:26:27	fce9b18c..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J36864_p2_exp_Kcycle_schwarz
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J36864_p2_exp_Kcycle_schwarz	04/15/2022 10:28:09	82b78cc9..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J65536_p2_exp_Kcycle_schwarz
Info: Found successful session "LinslvPerfSer	BottiPietroStokes2D-J65536_p2_exp_Kcycle_schwarz	04/15/2022 10:29:54	e44f7e0e..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: BottiPietroStokes2D-J147456_p2_exp_K

copied 50 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.

 Submitting: BottiPietroStokes2D-J36864_p2_pMultigrid
Deploying job BottiPietroStokes2D-J36864_p2_pMultigrid ... 
Deploying executables and additional files ...
Deployment directory: \\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022May16_124738
copied 50 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.

 Submitting: BottiPietroStokes2D-J65536_p2_pMultigrid
Deploying job BottiPietroStokes2D-J65536_p2_pMultigrid ... 
Deploying executables and additional files ...
Deployment directory: \\fdygitrunner\ValidationTests\LinslvPerfSer-XNSE_Solver2022May16_124931
copied 50 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.

 Submitting: BottiPietroStokes2D-J147456_p2_pMultigrid
Deploying job BottiPietroStokes2D-J147456_p2_pMultigrid ... 
Deploying executables and additional files ...
Deployment directory: \\fdygitrunner\Val

In [28]:
wmg.AllJobs

 ------------ MSHPC FailedOrCanceled; original Failed


#0: BottiPietroStokes2D-J64_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	BottiPietroStokes2D-J64_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#1: BottiPietroStokes2D-J256_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	BottiPietroStokes2D-J256_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#2: BottiPietroStokes2D-J576_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	BottiPietroStokes2D-J576_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)
#3: BottiPietroStokes2D-J1024_p2_direct_pardiso: FinishedSuccessful (MS HPC client  MSHPC-AllNodes @DC2, @\\fdygitrunner\ValidationTests)	BottiPietroStokes2D-J1024_p2_direct_pardiso: FinishedS

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions

In [ ]:
var NoSuccess = controls.Select(ctrl => ctrl.GetJob()).Where(job => job.Status != JobStatus.FinishedSuccessful).ToArray();
NoSuccess

In [ ]:
// In the case of some failed job, print the directory name for further inspection:
foreach(var fail in NoSuccess) {
    Console.WriteLine(fail + ":  @" + fail.LatestDeployment.DeploymentDirectory.FullName);
    //Console.WriteLine(fail.Stderr);
    //Console.WriteLine();
    //Console.WriteLine("***************************");
    //Console.WriteLine("***************************");
    //Console.WriteLine();
}

In [ ]:
/*
string PathOffset = @"C:\Users\jenkinsci\Desktop\LinSlvPerfFail-20apr22";
foreach(var fail in NoSuccess) {
    var C = fail.GetControl();
    C.savetodb = false;
    C.SaveToFile(System.IO.Path.Combine(PathOffset, fail.Name + ".obj"));
    
    string Stdout = fail.Stdout;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stdout.txt"), Stdout);
    
    string Stderr = fail.Stderr;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stderr.txt"), Stderr);
}
*/

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("BottiPietroStokes2D") && Si.SuccessfulTermination == false);
FailedSessions

In [ ]:
//FailedSessions.Select(si => (si, si.GetSessionDirectory(), si.DeployPath))

#### Asserting Success:

Remark: since this is currently (22 Apr. 2022) work-in-progress, we allow for some jobs to fail; At this intermediate milestone, I want to record (by the means of tests) what is already working. Thereby, I hope I won't break the working cases while trying to fix the failing ones.

In [ ]:
//foreach(var si in FailedSessions)
//   si.Delete(true);

In [ ]:
var prelim_allowedFails = new[] { "BottiPietroStokes2D-J2304_p2_pMultigrid", 
                                  "BottiPietroStokes2D-J16384_p2_pMultigrid", 
                                  "BottiPietroStokes2D-J36864_p2_pMultigrid", 
                                  "BottiPietroStokes2D-J65536_p2_pMultigrid", 
                                  "BottiPietroStokes2D-J147456_p2_pMultigrid", 
                                  "BottiPietroStokes2D-J36864_p3_pMultigrid", 
                                  "BottiPietroStokes2D-J16384_p5_pMultigrid", 
                                  "BottiPietroStokes2D-J36864_p5_pMultigrid" };

In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Where(job => !prelim_allowedFails.Contains(job.Name)).Count(), "Some Jobs Failed");

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Where(s => !prelim_allowedFails.Contains(s.Name)).Count(), "Some Sessions did not terminate successfully.");